### 9. Spark SQL

In this notebook we will look at Spark SQL. Spark SQL lets us work on structured data. The notebook will follow the contents of [Spark Documentation](https://spark.apache.org/docs/2.2.1/sql-programming-guide.html). With the extra information Spark has for the structure of the data, some additional optimizations can be performed.


#### Datasets and Dataframes

Dataset is a distrbuted collection of data. Dataset can be constructed from JVM objects and comes with the benefits of Strong typing and ability to use lambda functions of RDDs along with the optimization advantages of Spark SQL.

Dataframe is conceptually equivalent of a database table and has richer optimization under the hood. In Scala ``DataFrame`` is an alias of ``Dataset[Row]``

In [9]:
val spark = new org.apache.spark.sql.SQLContext(sc).sparkSession

First, we create an instance of ``org.apache.spark.sql.SparkSession`` from the available ``sc`` object of ``org.apache.spark.SparkContext``. To that that we need to instantiate ``org.apache.spark.sql.SQLContext`` from with the current ``sc`` variable and then get the ``sparkSession``

In [15]:
val peopledf = spark.read.json("people.json")
peopledf.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



What we saw above is loaded froma JSON file with contents 

```
{"name":"Michael"}
{"name":"Andy", "age":30}
{"name":"Justin", "age":19}

```

The file was downloaded from [this](https://github.com/apache/spark/blob/master/examples/src/main/resources/people.json) URL. The ``SparkSession`` instance was used to create a ``DataFrame`` instance from this JSON file and an appropriate type was inferred on loading the content. To view the schema of the `Dataframe` we do the following and we see that age is a numeric type and name is a string.

In [22]:
peopledf.printSchema

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)




All operations on ``DataFrame`` are untyped and they dont fail until we execute them. We will see some examples below of perfectly valid operations

In [34]:
import spark.implicits._

peopledf.select("name").show()
peopledf.select($"name", $"age" + 1).show()
peopledf.filter($"age" > 20).show()
peopledf.groupBy($"age").count().show()


+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Michael|     null|
|   Andy|       31|
| Justin|       20|
+-------+---------+

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

+----+-----+
| age|count|
+----+-----+
|  19|    1|
|null|    1|
|  30|    1|
+----+-----+



By importing ``import spark.implicits._`` we get to use ``$``. This is a convenient way to convert a string to a type ``org.apache.spark.sql.ColumnName``. The following two piece code fragments give the same results

``peopledf.filter(new org.apache.spark.sql.ColumnName("age") > 20).show()``

and

``peopledf.filter($"age" > 20).show()``

no prize for guessing which is more readable, especially if we want to specify multiple column names.

The ``select`` and ``filter`` operations returns us another``DataFrame``, however ``groupBy`` returns an object ``org.apache.spark.sql.RelationalGroupedDataset`` which further provides more aggregation operations like ``mean``, ``min``, ``max``, ``sum``, ``count``, ``pivot`` etc. 

The operations on ``DataFrames`` is not type safe and allows us to perfrom operations on types which doesn't make sense and gives unexpected results or even errors at run time. Following two examples, first where we compare a string value to be greater than a number giving us no results and second, we select a non existant field throwing a runtime exception.

In [54]:
peopledf.filter($"name" > 20).show()
peopledf.select($"test").show()

+---+----+
|age|name|
+---+----+
+---+----+



Name: org.apache.spark.sql.AnalysisException
Message: cannot resolve '`test`' given input columns: [age, name];;
'Project ['test]
+- AnalysisBarrier
      +- Relation[age#26L,name#27] json

StackTrace: 'Project ['test]
+- AnalysisBarrier
      +- Relation[age#26L,name#27] json

  at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
  at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$2.applyOrElse(CheckAnalysis.scala:88)
  at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$2.applyOrElse(CheckAnalysis.scala:85)
  at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$transformUp$1.apply(TreeNode.scala:289)
  at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$transformUp$1.apply(TreeNode.scala:289)
  at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:70)
  at org.apache.spark.sql.catalyst.trees.TreeNode.transformUp(Tree